In [1]:
import os
import sys

import torch

In [4]:
from __future__ import annotations

from typing import Any

class BaseMixTransform:
    def __init__(self, pre_transform=None, p=0.0)->None:
        self.p=p
        
    def __call__(self, labels: dict[str, Any]=None) -> dict[str, Any]:
        print('In base class')
        self._mix_transform(labels)
        
    def _mix_transform(self, labels: dict[str, Any]=None)->dict[str, Any]:
        raise NotImplementedError

class Mosaic(BaseMixTransform):

    def __init__(self, imgsz:int=640, p:float=1.0, n:int=4, buffer_enabled:bool=False):
        self.imgsz=imgsz
            
    def _mix_transform(self, labels: dict[str, Any]=None)->dict[str, Any]:
        print('Call a child-class function')
    
mosaic=Mosaic()
mosaic()

 

In base class
Call a child-class function


In [21]:
data_dirpath='D:/dev/video_processing/yolov11/notebook'
task_outs=torch.load(os.path.join(data_dirpath, 'nn.tasks.BaseModel.predict.pt'), weights_only=False)
pred_outs=torch.load(os.path.join(data_dirpath, 'engine.predictor.BasePredictor.stream_inference.pt'), weights_only=False)
post_ins=torch.load(os.path.join(data_dirpath, 'models.yolo.detect.predict.DetectionPredictor.postprocess.pt'), weights_only=False)
custom_outs=torch.load(os.path.join(data_dirpath, 'custom.pt'), weights_only=False)

In [22]:
[type(p) for p in task_outs], task_outs[0].shape, [type(p) for p in task_outs[-1]], [p.shape for p in task_outs[-1]]

([torch.Tensor, list],
 torch.Size([3, 84, 8400]),
 [torch.Tensor, torch.Tensor, torch.Tensor],
 [torch.Size([3, 144, 80, 80]),
  torch.Size([3, 144, 40, 40]),
  torch.Size([3, 144, 20, 20])])

In [23]:
[type(p) for p in pred_outs], pred_outs[0].shape, [type(p) for p in pred_outs[-1]] , [p.shape for p in pred_outs[-1]]

([torch.Tensor, list],
 torch.Size([3, 84, 8400]),
 [torch.Tensor, torch.Tensor, torch.Tensor],
 [torch.Size([3, 144, 80, 80]),
  torch.Size([3, 144, 40, 40]),
  torch.Size([3, 144, 20, 20])])

In [24]:
[type(p) for p in post_ins], post_ins[0].shape, [type(p) for p in post_ins[-1]] , [p.shape for p in post_ins[-1]]

([torch.Tensor, list],
 torch.Size([3, 84, 8400]),
 [torch.Tensor, torch.Tensor, torch.Tensor],
 [torch.Size([3, 144, 80, 80]),
  torch.Size([3, 144, 40, 40]),
  torch.Size([3, 144, 20, 20])])

In [25]:
[type(p) for p in custom_outs], custom_outs[0].shape, [type(p) for p in custom_outs[-1]] , [p.shape for p in custom_outs[-1]]

([torch.Tensor, list],
 torch.Size([3, 84, 8400]),
 [torch.Tensor, torch.Tensor, torch.Tensor],
 [torch.Size([3, 144, 80, 80]),
  torch.Size([3, 144, 40, 40]),
  torch.Size([3, 144, 20, 20])])

In [26]:
print(torch.allclose(task_outs[0], post_ins[0]), [torch.allclose(a,b) for a, b in zip(task_outs[-1], post_ins[-1])])
print(torch.allclose(custom_outs[0], post_ins[0]), [torch.allclose(a,b) for a, b in zip(custom_outs[-1], post_ins[-1])])

True [True, True, True]
False [False, False, False]


In [27]:
diff=(custom_outs[0]-post_ins[0]).abs()
print(diff.min().item(), diff.max().item())

0.0 411.9246826171875
